In [1]:
class Debug:
    def __init__(self, level):
        self.level = level
    def msg(self, level, m):
        if(level <= self.level):
            print(m)
debug = Debug(10).msg

In [2]:
%matplotlib inline
#!/usr/bin/python
#AWS redshift connector:
#setup
from datetime import date, timedelta
import pandas as pd
import random
import numpy as np

In [6]:
def uniqueSummary(csvname):
    df = pd.read_csv(csvname)
    newdf = pd.DataFrame(columns = df.columns)
    for colName in df.head():
        u = df[colName].unique()
        if len(u) > 100:
            newdf.loc[:, colName] = [len(u), "more than 100"]
        else:
            newdf.loc[:, colName] = [len(u), u]
    newdf = newdf.T
    
    newdf.to_csv("uniqueSummary_" + csvname)
        

In [7]:
with open("filenames.txt") as f:
    for i in f:
        i = i.strip('\n')
        uniqueSummary(i)
        

/home/r117/miniconda3/envs/snakes/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/r117/miniconda3/envs/snakes/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (3,26) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [9]:
nodedf = pd.read_csv("NODE.csv")
nodedf.columns

Index(['ACCIDENT_NO', 'NODE_ID', 'NODE_TYPE', 'AMG_X', 'AMG_Y', 'LGA_NAME',
       'Region Name', 'Deg Urban Name', 'Lat', 'Long', 'Postcode No'],
      dtype='object')

In [11]:
accidentlocdf = pd.read_csv("ACCIDENT_LOCATION.csv")
accidentlocdf.columns

Index(['ACCIDENT_NO', 'NODE_ID', 'ROAD_ROUTE_1', 'ROAD_NAME', 'ROAD_TYPE',
       'ROAD_NAME_INT', 'ROAD_TYPE_INT', 'DISTANCE_LOCATION',
       'DIRECTION_LOCATION', 'NEAREST_KM_POST', 'OFF_ROAD_LOCATION'],
      dtype='object')

In [13]:
print(nodedf.shape)
print(accidentlocdf.shape)

(145669, 11)
(144078, 11)


In [36]:
accidentlocdf.ROAD_ROUTE_1.head()

0    2090.0
1    5057.0
2    9999.0
3    2400.0
4    9999.0
Name: ROAD_ROUTE_1, dtype: float64

In [34]:
#check to see NODE_ID is not null in accident_location.csv
s = nodedf.shape[0]
compPoll = accidentlocdf.ACCIDENT_NO
rows = []
i = 0
new = nodedf[nodedf.loc[:, "ACCIDENT_NO"].isin(compPoll)]

    

(145669, 11)


In [35]:
print(len(nodedf.ACCIDENT_NO.unique()))
print(len(accidentlocdf.ACCIDENT_NO.unique()))


143179
144078


In [47]:
class LOC():
    def __init__(self,lat, long):
        self.lat = lat
        self.long = long

In [50]:
pointA = LOC(-37.8007939, 144.8803823)
pointB = LOC(-37.8622120,144.8831790)

In [54]:
def getNodes(A, B): #point A,B must be LOC class
    row = nodedf.shape[0] #row number
    rowcount = []
    for i in range(row):
        if(withinRange(nodedf.loc[i, 'Lat'], B.lat, A.lat) and 
           withinRange(nodedf.loc[i, 'Long'], A.long, B.long)):
            rowcount.append(i)
        else:
            continue
    print(len(rowcount))
def withinRange(loc1,loc2, loc3):
    if loc2 <= loc1 <= loc3:
        return True
    else:
        return False

In [55]:
getNodes(pointA, pointB)

248
